# Pair programming API
Ejercicios de API's: Enunciado - Obtener datos energéticos usando la API de Red Eléctrica. Descripción: La empresa para la que trabajamos dispone de una base de datos que contiene información sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde 1998 hasta 2022. El objetivo es enriquecer estos datos con información sobre la generación de energía renovable y no renovable, utilizando la API de REData, la cual proporciona información sobre el mercado energético en España. Puedes encontrar la documentación de la API .

1. Extracción de Datos a Nivel Nacional: Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. La información debe ser recopilada a nivel mensual. Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable. Almacenar los datos en un DataFrame con la siguiente estructura:

In [1]:
# Librerias para exploración y trasnformación de datos
import pandas as pd
import requests
import re 
import json

In [2]:
# Configuración para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_columns', None) 

In [3]:
# Definir la url
url_energia = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=month"

# Realiza una solicitud GET a la URL proporcionada
datos_llamada_energia = requests.get(url_energia)

In [4]:
# Llamada y estado 
datos_llamada_energia.status_code
print("La respuesta es:", datos_llamada_energia)

La respuesta es: <Response [200]>


In [5]:
# Paso a formato json
datos_llamada_energia.json()

{'data': {'type': 'Generación por tecnología',
  'id': 'gen1',
  'attributes': {'title': 'Generación por tecnología',
   'last-update': '2019-06-12T15:32:24.000+02:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'MISS'}}},
 'included': [{'type': 'Hidráulica',
   'id': '10288',
   'groupId': '1',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2019-06-12T15:32:24.000+02:00',
    'values': [{'value': 5119512.38,
      'percentage': 0.19757364151478785,
      'datetime': '2011-01-01T00:00:00.000+01:00'},
     {'value': 2998051.279,
      'percentage': 0.12956959110916957,
      'datetime': '2011-02-01T00:00:00.000+01:00'},
     {'value': 3910363.276,
      'percentage': 0.15594663753166463,
      'datetime': '2011-03-01T00:00:00.000+01:00'},
     {'value': 3537937.995,
      'percentage': 0.16314748356694456,
      'datetime': '2011-04-0

In [6]:
# Almacenar json en una variable
energias = datos_llamada_energia.json()

len(energias)

2

In [7]:
energias.keys()

dict_keys(['data', 'included'])

In [8]:
energias["data"]

{'type': 'Generación por tecnología',
 'id': 'gen1',
 'attributes': {'title': 'Generación por tecnología',
  'last-update': '2019-06-12T15:32:24.000+02:00',
  'description': None},
 'meta': {'cache-control': {'cache': 'MISS'}}}

In [9]:
energias["included"]

[{'type': 'Hidráulica',
  'id': '10288',
  'groupId': '1',
  'attributes': {'title': 'Hidráulica',
   'description': None,
   'color': '#0090d1',
   'type': 'Renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2019-06-12T15:32:24.000+02:00',
   'values': [{'value': 5119512.38,
     'percentage': 0.19757364151478785,
     'datetime': '2011-01-01T00:00:00.000+01:00'},
    {'value': 2998051.279,
     'percentage': 0.12956959110916957,
     'datetime': '2011-02-01T00:00:00.000+01:00'},
    {'value': 3910363.276,
     'percentage': 0.15594663753166463,
     'datetime': '2011-03-01T00:00:00.000+01:00'},
    {'value': 3537937.995,
     'percentage': 0.16314748356694456,
     'datetime': '2011-04-01T00:00:00.000+02:00'},
    {'value': 2797500.027,
     'percentage': 0.1279110558374734,
     'datetime': '2011-05-01T00:00:00.000+02:00'},
    {'value': 2070305.344,
     'percentage': 0.09182271836090777,
     'datetime': '2011-06-01T00:00:00.000+02:00'},
    {'value': 1661

In [10]:
diccionario_energia = {"tipo_energia": [], "tipo_generacion": [], "fecha_lista": [], "valor_lista": []}

included = datos_llamada_energia.json()

datos = included["included"]

In [11]:
def extraer_datos_energia(datos_llamada_energia, diccionario_energia):
    
    lista_energia = []
    
    for dato in datos:
        tipo_energia = dato["type"]
        tipo_generacion = dato["attributes"]["type"]
        fecha_lista = dato["attributes"]["values"][0]["datetime"]
        valor_lista = dato["attributes"]["values"][0]["value"]
    
        lista_energia.append({"tipo_energia": tipo_energia, "tipo_generacion": tipo_generacion, "fecha_lista": fecha_lista, "valor_lista": valor_lista})
        
    # Crear un Dataframe a partir del diccinario de datos
    df_lista_energia = pd.DataFrame(lista_energia)
    return df_lista_energia

df_energia = extraer_datos_energia(datos_llamada_energia, diccionario_energia)
df_energia.head()

,tipo_energia,tipo_generacion,fecha_lista,valor_lista
0,Hidráulica,Renovable,2011-01-01T00:00:00.000+01:00,5119512.380
1,Turbinación bombeo,No-Renovable,2011-01-01T00:00:00.000+01:00,283944.912
2,Nuclear,No-Renovable,2011-01-01T00:00:00.000+01:00,4706500.878
3,Carbón,No-Renovable,2011-01-01T00:00:00.000+01:00,2303087.707
4,Fuel + Gas,No-Renovable,2011-01-01T00:00:00.000+01:00,-751.170


In [12]:
df_energia

,tipo_energia,tipo_generacion,fecha_lista,valor_lista
0,Hidráulica,Renovable,2011-01-01T00:00:00.000+01:00,5.119512e+06
1,Turbinación bombeo,No-Renovable,2011-01-01T00:00:00.000+01:00,2.839449e+05
2,Nuclear,No-Renovable,2011-01-01T00:00:00.000+01:00,4.706501e+06
3,Carbón,No-Renovable,2011-01-01T00:00:00.000+01:00,2.303088e+06
4,Fuel + Gas,No-Renovable,2011-01-01T00:00:00.000+01:00,-7.511700e+02
5,Motores diésel,No-Renovable,2011-01-01T00:00:00.000+01:00,3.035381e+05
6,Turbina de gas,No-Renovable,2011-01-01T00:00:00.000+01:00,5.150644e+04
7,Turbina de vapor,No-Renovable,2011-01-01T00:00:00.000+01:00,2.391522e+05
8,Ciclo combinado,No-Renovable,2011-01-01T00:00:00.000+01:00,5.349870e+06
9,Eólica,Renovable,2011-01-01T00:00:00.000+01:00,4.033540e+06


In [ ]:
# Ejercicio realizado con los datos del 2011!

In [18]:
diccionario_energia = {"tipo_de_energia": [], "tipo_generacion": [], "fecha": [], "valor": []}

for año in range(2011, 2023):
    url = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=month&geo"
    llamada= requests.get(url)
    llamada.status_code

    llamada_json = llamada.json()

    for elemento in llamada_json["included"]:
        energia = elemento["type"]
        generacion = elemento["attributes"]["type"]
        
        
        for e in range(0,len(elemento["attributes"]["values"])):
            fecha = elemento["attributes"]["values"][e]["datetime"]  
            valor = elemento["attributes"]["values"][e]["value"]  
            diccionario_energia["tipo_de_energia"].append(energia)
            diccionario_energia["tipo_generacion"].append(generacion)
            diccionario_energia["fecha"].append(fecha)
            diccionario_energia["valor"].append(valor)
            
print(diccionario_energia)

{'tipo_de_energia': ['Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Hidráulica', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Turbinación bombeo', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Nuclear', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Carbón', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Fuel + Gas', 'Motores diésel', 'Motores diésel', 'Motores diésel', 'Motores diésel', 'Motores diésel', 'Motores diésel', 'Motores diésel', '

In [16]:
df_energia = pd.DataFrame(diccionario_energia)
df_energia

,tipo_de_energia,tipo_generacion,fecha,valor
0,Hidráulica,Renovable,2011-01-01T00:00:00.000+01:00,5.119512e+06
1,Hidráulica,Renovable,2011-02-01T00:00:00.000+01:00,2.998051e+06
2,Hidráulica,Renovable,2011-03-01T00:00:00.000+01:00,3.910363e+06
3,Hidráulica,Renovable,2011-04-01T00:00:00.000+02:00,3.537938e+06
4,Hidráulica,Renovable,2011-05-01T00:00:00.000+02:00,2.797500e+06
...,...,...,...,...
2482,Generación total,Generación total,2022-08-01T00:00:00.000+02:00,2.528589e+07
2483,Generación total,Generación total,2022-09-01T00:00:00.000+02:00,2.366621e+07
2484,Generación total,Generación total,2022-10-01T00:00:00.000+02:00,2.195664e+07
2485,Generación total,Generación total,2022-11-01T00:00:00.000+01:00,2.152262e+07


In [ ]:
# ELIMINARRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
# Ejercicio realizado para el rango de 2011 a 2023:
# Iterar sobre los años y meses:

# for year in range(2011, 2023):
#     for mes in range(1,13):
#         last_day = 31 if mes in [1, 3, 5, 7, 8, 10, 12] else 30 if mes in [4, 6, 9, 11] else 28 if year % 4 != 0 or (year % 100 == 0 and year % 400 != 0) else 29
        
#         start_date = f"{year}-{mes:02d}-01"
#         end_date = f'{year}-{mes:02d}-31' 
#         # Llammamos a la API
#         url_energia2 = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={fecha_inicio}T00:00&end_date={fecha_fin}T23:59&time_trunc=month"
#         energiaa = llamar_api(url_energia2, start_date, end_date)
        
#         if energiaa:
#             extraccion_datos(energias, diccionario_energia)
         
#         for energia in data_json['included']:
#             tipo_energia = energia['attributes']['title']
#             tipo_generacion = energia['attributes']['type']
#             fecha = energia['attributes']['values'][0]['datetime']
#             valor = energia['attributes']['values'][0]['value']
            
# #Apendeamos en las listas vacías del diccionario 
#             diccionario_energia['tipo_energia'].append(tipo_energia)
#             diccionario_energia['tipo_generacion'].append(tipo_energia)
#             diccionario_energia['fecha'].append(fecha)
#             diccionario_energia['valor'].append(valor)

KeyError: 'fecha'

2. Extracción de Datos por Comunidad Autónoma: Extraer datos de generación de energía renovable y no renovable por Comunidad Autónoma. Utilizar el diccionario cod_comunidades para obtener los códigos de cada comunidad autónoma. Transformar los datos para incluir una columna que especifique la comunidad autónoma y su identificador. Almacenar los datos en un DataFrame con la siguiente estructura:

In [24]:
# El diccionario de cod_comunidades es:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [25]:
# Creamos una variable para almancenar los datos vacia
energia_ccaa = {'tipo_energia': [], 'tipo_generacion': [], 'fecha': [], 'valor': [], 'ccaa': []}

In [26]:
for comunidad in cod_comunidades:    
    for año in range(2011, 2023):
        url_ccaa = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={cod_comunidades[comunidad]}"
        llamadaenergia = requests.get(url_ccaa)
        llamadaenergia.status_code

        llamada_json = llamada.json()

        for elemento in llamada_json["included"]:
            energia = elemento["type"]
            generacion = elemento["attributes"]["type"]
        
            for e in range(0,len(elemento["attributes"]["values"])):
                fecha = elemento["attributes"]["values"][e]["datetime"]  
                valor = elemento["attributes"]["values"][e]["value"]  
                energia_ccaa["tipo_energia"].append(energia)
                energia_ccaa["tipo_generacion"].append(generacion)
                energia_ccaa["fecha"].append(fecha)
                energia_ccaa["valor"].append(valor)
                energia_ccaa["ccaa"].append(comunidad)

In [28]:
# Convertimos el diccionario de comunidades autónomas en un DataFrame
df_energia_comunidades = pd.DataFrame(energia_ccaa)

# Imprimimos el DataFrame de las comunidades
df_energia_comunidades

,tipo_energia,tipo_generacion,fecha,valor,ccaa
0,Hidráulica,Renovable,2022-01-01T00:00:00.000+01:00,2.085857e+06,Ceuta
1,Hidráulica,Renovable,2022-02-01T00:00:00.000+01:00,1.174970e+06,Ceuta
2,Hidráulica,Renovable,2022-03-01T00:00:00.000+01:00,1.775478e+06,Ceuta
3,Hidráulica,Renovable,2022-04-01T00:00:00.000+02:00,1.802752e+06,Ceuta
4,Hidráulica,Renovable,2022-05-01T00:00:00.000+02:00,1.923715e+06,Ceuta
...,...,...,...,...,...
46963,Generación total,Generación total,2022-08-01T00:00:00.000+02:00,2.528589e+07,La Rioja
46964,Generación total,Generación total,2022-09-01T00:00:00.000+02:00,2.366621e+07,La Rioja
46965,Generación total,Generación total,2022-10-01T00:00:00.000+02:00,2.195664e+07,La Rioja
46966,Generación total,Generación total,2022-11-01T00:00:00.000+01:00,2.152262e+07,La Rioja


In [ ]:
#ELIMINARRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
# for comunidad in cod_comunidades.item():
#     for año in range(2011,2023):
#         url_ccaa = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=month&geo
#         print(url_ccaa)
        
#         energia = requests.get(url)
        
#         energia_json = energia.json()
        
#         for info in energia_json

#  # Convertimos el diccionario de comunidades autónomas en un DataFrame
# df_energia_comunidades = consultar_datos(diccionario_comunidades)

# # Ahora imprimimos el DataFrame de las comunidades
# print(df_energia_comunidades)

# print('--------------------------------------')

# df_energia_comunidades

In [ ]:
# ELIMINARRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR CONVERSACION CON SAIRA 
# for k,v in cod_comunidades.items():
#     for anyo in range(2011,2023):
#         for mes in range(1, 13):
#             fecha_inicio = f"{anyo}-{mes:02d}-01"
#             fecha_fin = f'{anyo}-{mes:02d}-31'
            
#         url_ccaa = f'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={anyo}-01-01T00:00&end_date={anyo}-12-31T23:59&time_trunc=month&&geo_limit=ccaa&geo_ids={v}'
#         print(url_ccaa)
        
#         energia = requests.get(url)
#         energia_json = energia.json()
        
#         energia_comunidades = llamar_api(url_ccaa, fecha_inicio, fecha_fin)
        
#         if energia_comunidades:
#             for info in energia_json['included']:
#                 tipo_energia = info["type"]
#                 tipo_generacion = info['attributes']['type']
#                 fecha = info['attributes']['values'][0]['datetime']
#                 valor = info['attributes']['values'][0]['value']
#                 energia_ccaa["tipo_energia"].append(tipo_energia)
#                 energia_ccaa["tipo_generacion"].append(tipo_generacion)
#                 energia_ccaa['fecha'].append(fecha)
#                 energia_ccaa['valor'].append(valor)
#                 energia_ccaa['ccaa'].append(k)

https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=month&&geo_limit=ccaa&geo_ids=8744


NameError: name 'url' is not defined